In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy.fft import rfft,rfftfreq

sns.set()

In [ ]:

SAMPLE_RATE = 16  # Hertz - Nyquist theorem : sampling rate at least 2 x highest signal freq. 
DURATION = 5  # Seconds

def generate_sine_wave(freq, sample_rate, duration):
    x = np.linspace(0, duration, sample_rate * duration, endpoint=False)
    frequencies = x * freq
    # 2pi because np.sin takes radians
    y = np.sin((2 * np.pi) * frequencies)
    return x, y

# Generate a 2 hertz sine wave that lasts for 5 seconds
x, y = generate_sine_wave(2, SAMPLE_RATE, DURATION)
x2,y2 = generate_sine_wave(4,SAMPLE_RATE,DURATION)

plt.figure(figsize=(18,12))
plt.plot(x, y)
plt.plot(x2,y2)
plt.show()

In [ ]:
signal = y + y2 * 0.5

N = SAMPLE_RATE * DURATION

yf = rfft(signal)
xf = rfftfreq(N, 1 / SAMPLE_RATE)

plt.plot(xf, np.abs(yf) / N)
plt.ylabel('power')
plt.xlabel('frequency')
plt.show()

In [ ]:
faked_yearly_data = np.array([0,1,2,3,4,4,3,2,1,0] * 10)
faked_yearly_data = faked_yearly_data / faked_yearly_data.max() #normalize

plt.plot(faked_yearly_data,'o--')

In [ ]:

sample_rate = 10 # 10 samples per decennium

yf = rfft(faked_yearly_data)
xf = rfftfreq(len(faked_yearly_data),1 / sample_rate) #normalize so that we can have low sampling rate (Nyquist)

plt.plot(xf,np.abs(yf) / len(faked_yearly_data))
plt.xlabel('freq [ 1/ decade]')
